# Recommandation de reviews similaires

In [1]:
import pandas as pd
import numpy as np
import re
import os
import html
import gensim

data_dir = "../data/"

In [2]:
reviews_interstellar = pd.read_csv(data_dir+"raw/interstellar_critique.csv")
reviews_fightclub = pd.read_csv(data_dir+"raw/fightclub_critiques.csv")

In [3]:
reviews_interstellar.describe()

,id,rating,review_hits,gen_review_like_count,user_id
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,4.896581e+07,7.651000,793.829000,10.377000,2.052111e+05
std,2.650888e+07,2.125492,1793.170913,38.243397,1.929741e+05
min,5.968200e+06,1.000000,244.000000,0.000000,3.020000e+02
25%,4.090442e+07,6.000000,330.000000,0.000000,6.142350e+04
50%,4.156629e+07,8.000000,406.000000,2.000000,1.583265e+05
75%,4.623296e+07,9.000000,595.250000,5.000000,3.049102e+05
max,2.569371e+08,10.000000,22676.000000,513.000000,1.669513e+06


In [4]:
reviews_fightclub.describe()

,id,rating,review_hits,gen_review_like_count,user_id
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,1.048852e+08,8.155000,431.733000,4.736000,6.784574e+05
std,1.122182e+08,2.193579,1516.896269,18.003512,8.001516e+05
min,7.120000e+02,1.000000,4.000000,0.000000,1.190000e+02
25%,9.223028e+06,8.000000,52.000000,0.000000,4.008500e+04
50%,4.366115e+07,9.000000,220.000000,1.000000,2.502235e+05
75%,2.134670e+08,10.000000,406.000000,2.000000,1.305649e+06
max,3.281070e+08,10.000000,35817.000000,292.000000,2.376997e+06


In [5]:
reviews_interstellar.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"<p>Chers lecteurs, chères lectrices,</p>\n\n<p...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,<p>Malgré ce que j'entends dire ou lis sur le ...,Tous les chemins mènent à l'Homme,347,blig,133793
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"<p>Un grand film, pour moi, c'est un film qui ...",Rage against the dying of the light.,453,Samu-L,41951
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,<p>Tout en n'étant absolument pas un admirateu...,Inspacetion,273,Nushku,24046


In [6]:
reviews_fightclub.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721


## Nettoyage des données

In [7]:
reviews_interstellar.isnull().sum()

id                           0
URL                          0
rating                       0
review_date_creation         0
review_date_last_update      0
review_hits                  0
review_content               0
review_title               168
gen_review_like_count        0
username                     0
user_id                      0
dtype: int64

In [8]:
reviews_fightclub.isnull().sum()

id                           0
URL                          0
rating                       0
review_date_creation         0
review_date_last_update      0
review_hits                  0
review_content               1
review_title               351
gen_review_like_count        0
username                     0
user_id                      0
dtype: int64

On s'intéresse au contenu des reviews donc celles sans titre peuvent être gardées. Par contre la review sans 'content' sera effacée.

In [9]:
reviews_fightclub.dropna(subset="review_content", inplace=True)
reviews_fightclub.reset_index(drop=True, inplace=True)
reviews_fightclub.isnull().sum()

id                           0
URL                          0
rating                       0
review_date_creation         0
review_date_last_update      0
review_hits                  0
review_content               0
review_title               350
gen_review_like_count        0
username                     0
user_id                      0
dtype: int64

In [10]:
print("Nombre de doublons dans reviews_interstellar :", reviews_interstellar.duplicated().sum())
print("Nombre de doublons dans reviews_fightclub :", reviews_fightclub.duplicated().sum())

Nombre de doublons dans reviews_interstellar : 0
Nombre de doublons dans reviews_fightclub : 0


In [11]:
def text_cleaning(s):
    res = html.unescape(s)
    res = re.sub("<.*?>", "", res)
    res = re.sub("[\n\t\r]", "", res)
    
    return res

In [12]:
reviews_interstellar['review_content'] = reviews_interstellar['review_content'].apply(text_cleaning)
reviews_interstellar.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"Chers lecteurs, chères lectrices,Avant d'entam...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,Malgré ce que j'entends dire ou lis sur le sit...,Tous les chemins mènent à l'Homme,347,blig,133793
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"Un grand film, pour moi, c'est un film qui m'e...",Rage against the dying of the light.,453,Samu-L,41951
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,Tout en n'étant absolument pas un admirateur d...,Inspacetion,273,Nushku,24046


In [13]:
reviews_fightclub['review_content'] = reviews_fightclub['review_content'].apply(text_cleaning)
reviews_fightclub.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"Qu'une oeuvre aussi folle, aussi inconfortable...",Sons of Anarchy.,199,Gand-Alf,55207
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,Beaucoup pensent que ce film n'est qu'un éloge...,Je suis l'ego démesuré de Jack,191,Ano,2333
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"Objet d’un véritable culte, Fight Club a marqu...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721


## Création du dataset

On va fusionner les deux dataframes pour en avoir un avec les deux films. Mais avant ça on va créer une nouvelle colonne avec le titre du film afin de pouvoir filtrer les recommandations.

In [14]:
reviews_interstellar['movie_title'] = 'Interstellar'
reviews_interstellar.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916,Interstellar
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"Chers lecteurs, chères lectrices,Avant d'entam...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882,Interstellar
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,Malgré ce que j'entends dire ou lis sur le sit...,Tous les chemins mènent à l'Homme,347,blig,133793,Interstellar
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"Un grand film, pour moi, c'est un film qui m'e...",Rage against the dying of the light.,453,Samu-L,41951,Interstellar
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,Tout en n'étant absolument pas un admirateur d...,Inspacetion,273,Nushku,24046,Interstellar


In [15]:
reviews_fightclub['movie_title'] = 'Fight Club'
reviews_fightclub.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316,Fight Club
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552,Fight Club
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"Qu'une oeuvre aussi folle, aussi inconfortable...",Sons of Anarchy.,199,Gand-Alf,55207,Fight Club
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,Beaucoup pensent que ce film n'est qu'un éloge...,Je suis l'ego démesuré de Jack,191,Ano,2333,Fight Club
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"Objet d’un véritable culte, Fight Club a marqu...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721,Fight Club


In [16]:
content_dataset = pd.concat([reviews_interstellar, reviews_fightclub], axis=0)
content_dataset.reset_index(drop=True, inplace=True)
content_dataset

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916,Interstellar
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"Chers lecteurs, chères lectrices,Avant d'entam...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882,Interstellar
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,Malgré ce que j'entends dire ou lis sur le sit...,Tous les chemins mènent à l'Homme,347,blig,133793,Interstellar
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"Un grand film, pour moi, c'est un film qui m'e...",Rage against the dying of the light.,453,Samu-L,41951,Interstellar
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,Tout en n'étant absolument pas un admirateur d...,Inspacetion,273,Nushku,24046,Interstellar
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,297093474,https://senscritique.com/film/xxx/critique/297...,10,14/11/23 14:47,14/11/23 14:47,4,"Bonjour, Regardez ce film car : oui. Excellent...",Ceci n’est pas une critique constructive,0,Armequin,2183533,Fight Club
1995,205504617,https://senscritique.com/film/xxx/critique/205...,5,13/08/25 16:54,13/08/25 16:54,4,"Ce film est très surcoté, il as tout pour etre...",fantasme bourgeois de rage auto destructrice ....,1,Gorgondounet,1426061,Fight Club
1996,318641521,https://senscritique.com/film/xxx/critique/318...,10,17/03/25 14:59,17/03/25 14:59,4,Banger,NaN,0,charlottep2b,2330845,Fight Club
1997,325535346,https://senscritique.com/film/xxx/critique/325...,8,11/06/25 17:00,11/06/25 21:38,4,https://boxd.it/5UblLD,"De base on ne parle pas du ***** ****, mais bo...",0,GabrielHut,2358467,Fight Club


In [17]:
output_path = data_dir + "processed/content_dataset.csv"

os.makedirs(os.path.dirname(output_path), exist_ok=True)
content_dataset.to_csv(output_path)

## Word2Vec

On crée les tokens d'abord.

In [18]:
tokens = content_dataset['review_content'].apply(gensim.utils.simple_preprocess)

In [19]:
tokens

0       [aïe, aïe, aïe, nous, voilà, interstellar, le,...
1       [chers, lecteurs, chères, lectrices, avant, en...
2       [malgré, ce, que, entends, dire, ou, lis, sur,...
3       [un, grand, film, pour, moi, est, un, film, qu...
4       [tout, en, étant, absolument, pas, un, admirat...
                              ...                        
1994    [bonjour, regardez, ce, film, car, oui, excell...
1995    [ce, film, est, très, surcoté, il, as, tout, p...
1996                                             [banger]
1997                             [https, boxd, it, ublld]
1998    [nous, ne, sommes, que, des, produits, de, con...
Name: review_content, Length: 1999, dtype: object

In [20]:
model = gensim.models.Word2Vec(tokens, vector_size=300, window= 5 , min_count= 2)

In [21]:
def get_text_vector(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [22]:
from tqdm import tqdm
embeddings = np.vstack([
        get_text_vector(sentence, model) for sentence in tqdm(tokens)
    ])

100%|██████████| 1999/1999 [00:00<00:00, 2240.93it/s]


In [23]:
embeddings[0]

array([ 4.64632474e-02,  2.10690662e-01, -5.07491231e-02, -1.54003739e-01,
       -9.71432179e-02, -2.39203379e-01,  3.10286462e-01,  4.85420078e-01,
       -1.52114807e-02, -2.73439467e-01,  2.03347355e-01, -1.36532977e-01,
       -1.02431081e-01,  9.92539451e-02, -1.30967259e-01, -2.70388931e-01,
        2.70812422e-01, -4.52776588e-02, -1.19833007e-01, -1.75531343e-01,
        4.35766838e-02, -9.50804204e-02,  1.33967340e-01, -6.39345963e-03,
        1.41894057e-01, -2.30004162e-01, -4.16337729e-01, -5.24095818e-02,
       -2.02859879e-01, -3.34253639e-01,  5.06376550e-02, -1.19922057e-01,
        1.49934009e-01, -1.09680392e-01,  8.45690817e-02,  1.81536794e-01,
        9.73582119e-02, -3.43926996e-01, -8.86325389e-02, -1.24776419e-02,
       -1.11417197e-01,  8.61317590e-02, -7.89180845e-02, -2.50195473e-01,
        1.68999881e-01,  1.79046988e-01,  2.93614715e-02,  1.01771310e-01,
        1.49502501e-01,  2.40758806e-01, -1.28987074e-01,  4.47289348e-02,
       -8.02257806e-02,  

In [24]:
content_dataset['review_embedding'] = embeddings.tolist()
content_dataset.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title,review_embedding
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916,Interstellar,"[0.0464632473886013, 0.2106906622648239, -0.05..."
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"Chers lecteurs, chères lectrices,Avant d'entam...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882,Interstellar,"[0.09056264907121658, 0.1640280783176422, 0.00..."
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,Malgré ce que j'entends dire ou lis sur le sit...,Tous les chemins mènent à l'Homme,347,blig,133793,Interstellar,"[0.046766310930252075, 0.19820988178253174, -0..."
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"Un grand film, pour moi, c'est un film qui m'e...",Rage against the dying of the light.,453,Samu-L,41951,Interstellar,"[0.10489460080862045, 0.19818754494190216, -0...."
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,Tout en n'étant absolument pas un admirateur d...,Inspacetion,273,Nushku,24046,Interstellar,"[0.04001280292868614, 0.20974613726139069, -0...."


In [25]:
content_dataset.head()

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title,review_embedding
0,25246858,https://senscritique.com/film/xxx/critique/252...,10,06/11/14 12:29,06/11/14 12:29,22676,"Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...","All you need is love, love, love, love...",513,Kobayashhi,44916,Interstellar,"[0.0464632473886013, 0.2106906622648239, -0.05..."
1,40035436,https://senscritique.com/film/xxx/critique/400...,4,15/11/14 13:13,15/11/14 13:13,20186,"Chers lecteurs, chères lectrices,Avant d'entam...",10 « bonnes » raisons relatives de ne pas aime...,278,Veather,181882,Interstellar,"[0.09056264907121658, 0.1640280783176422, 0.00..."
2,39231131,https://senscritique.com/film/xxx/critique/392...,10,28/02/15 10:36,28/02/15 10:36,19215,Malgré ce que j'entends dire ou lis sur le sit...,Tous les chemins mènent à l'Homme,347,blig,133793,Interstellar,"[0.046766310930252075, 0.19820988178253174, -0..."
3,26720711,https://senscritique.com/film/xxx/critique/267...,9,06/11/14 23:30,06/11/14 23:30,19157,"Un grand film, pour moi, c'est un film qui m'e...",Rage against the dying of the light.,453,Samu-L,41951,Interstellar,"[0.10489460080862045, 0.19818754494190216, -0...."
4,24990038,https://senscritique.com/film/xxx/critique/249...,5,05/11/14 20:44,05/11/14 20:44,16856,Tout en n'étant absolument pas un admirateur d...,Inspacetion,273,Nushku,24046,Interstellar,"[0.04001280292868614, 0.20974613726139069, -0...."


In [41]:
review_id = content_dataset['id'][15]
review_index = np.where(content_dataset['id'] == review_id)[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommandation_reviews(review_id, content_dataset, k=5):

    review_index = np.where(content_dataset['id'] == review_id)[0][0]
    
    movie_title = content_dataset['movie_title'][review_index]
    review_embedding = content_dataset['review_embedding'][review_index]

    sub_content = content_dataset[(content_dataset['movie_title']==movie_title) & (content_dataset.index != review_index)].copy()
    
    sim_scores = cosine_similarity([review_embedding], sub_content['review_embedding'].tolist())[0]
    top_k_index = np.argsort(sim_scores)[-k:][::-1]

    top_k_reviews = sub_content.iloc[top_k_index]

    return top_k_reviews, sim_scores[:k]

In [59]:
print(content_dataset['review_content'][review_index])

Nous ne sommes que des produits de consommation digérés en illusion de nous même. Société aliénante, lobotomisée par l'illusion du choix. 


In [60]:
reviews, sim_scores = recommandation_reviews(review_id, content_dataset, 5)
print(f"Recommandation de reviews pour le film : {content_dataset['movie_title'][review_index]}\n")
for r in reviews['review_content'] :
    print(r)

Index([1998], dtype='int64')


ValueError: Can only compare identically-labeled Series objects

In [ ]:
review_id = content_dataset['id'][1998]

In [50]:
reviews, sim_scores = recommandation_reviews(review_id, content_dataset, 5)
print(f"Recommandation de reviews pour le film : {content_dataset['movie_title'][1998]}\n")
for r in reviews['review_content'] :
    print(r)  

1998
Recommandation de reviews pour le film : Fight Club

L'histoire démarre avec des faits anodins, des petits drames de la vie de tout les jours. De fil en aiguille la trame s'intensifie autour du "Fight club" : nous sommes pointés du doigt et mis au défi. Le narrateur est tout simplement génial, la mise en scène de ses pensées, de ses idées ainsi que les fruits de son imagination sont parfaitement maitrisés, ils nous nous laissent perplexes mais nous fascinent. L'effet souhaité est là. Le "Fight club", j'aimerais en parler ... Mais les deux premières règles m'interdisent d'en parler.Le film est une critique de la société qui nous manipule, des publicitaires qui nous poussent à la consommation, de l'argent qui appelle l'argent, des statuts professionnels qui condamnent notre personne ainsi que celles des clients... Autant de visons qui noircissent le cœur de notre narrateur, mélange de paranoïa et d'amères vérités. La question qui tarabuste notre personnage principal est donc bien "Q